In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
import statsmodels.formula.api as smf
from fixedeffect.fe import fixedeffect, did, getfe


In [300]:
data = pd.read_csv('pset2.csv')

In [301]:
data['log_fatal_per_cap'] = np.log(data['fatalities']/data['population'])



#some data "cleaning"
data['years_post_prim'] = data['year']
df = pd.DataFrame(columns = data.columns)
data['never_treat'] = 0
for s in sorted(list(set(list(data['state'])))):
    x = 0
    dat = data[data['state']==s]
    for y in list(set(list(data['year']))):
        if dat.iloc[22]['primary']== 0:
            dat['years_post_prim'] = dat['years_post_prim'] - 2004
            #print(dat['years_post_prim'])
            df = df.append(dat)
            break
        if dat.iloc[x]['primary']==1:
            dat['years_post_prim'] = dat['years_post_prim'] - y
            dat['never_treat'] = 1
            df = df.append(dat)
            break
        x = x+1
           
df['post'] = df['years_post_prim']
orig = list(set(list(df['years_post_prim'])))
new = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

df['post'] = df['post'].replace(orig,new)

<ipython-input-301-91911ea80437>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['years_post_prim'] = dat['years_post_prim'] - y
<ipython-input-301-91911ea80437>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['never_treat'] = 1
<ipython-input-301-91911ea80437>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [302]:
print(len(sorted(list(set(list(data['state']))))))
print(list(set(list(data['year']))))

48
[1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003]


In [306]:
#1

tau_list=[]
cohort_size = []
for year in list(set(list(data['year'])))[1:]:
    dat = df[df['year']==year]
    dat2 = df[df['year']== year-1]
    
    mean_treat = dat[dat['years_post_prim']==0]['log_fatal_per_cap'].mean()
    mean_no_treat = dat[dat['years_post_prim']<0]['log_fatal_per_cap'].mean()
    mean_treat2 = dat2[dat2['years_post_prim']==-1]['log_fatal_per_cap'].mean()
    mean_no_treat2 = dat2[dat2['years_post_prim']<-1]['log_fatal_per_cap'].mean()
    
    tau = mean_treat - mean_treat2 -mean_no_treat +mean_no_treat2

    cohort = dat[dat['years_post_prim']==0]['log_fatal_per_cap'].shape[0] 
    cohort_size = dat[dat['years_post_prim']<0]['log_fatal_per_cap'].shape[0] + cohort
    tau_list.append((tau, cohort, cohort_size))
    
tau_list = [tau for tau in tau_list if str(tau[0]) != 'nan']
print(tau_list)

[(-0.019374693419648148, 1, 48), (-0.10405106982486512, 4, 47), (-0.04644978030617852, 1, 43), (-0.1317145516344156, 1, 42), (-0.010687478133418082, 1, 41), (0.03747439095362326, 2, 40), (-0.008891234326839026, 3, 38), (-0.02545566577520031, 3, 35), (-0.026686766476580415, 1, 32), (0.13967986498427098, 1, 31)]


In [309]:
tau_weighted = 0
total = 0
for tau in tau_list:
    total = total + tau[1]
    tau_weighted = tau_weighted + (tau[0]*tau[1])/tau[2]
print(tau_weighted)
print(total)

-0.01107430161452777
18


In [316]:
data_no_treat = df[df['years_post_prim']<0]
x1 = pd.get_dummies(data_no_treat['state'])
x2 = pd.get_dummies(data_no_treat['year'])
data_no = pd.concat([data_no_treat, x1, x2], axis=1)

x3 = sorted(list(set(list(data_no_treat['state']))))[1:]
x4 = sorted(list(set(list(data_no_treat['year']))))[1:]


vals = sm.OLS(data_no['log_fatal_per_cap'], sm.add_constant(data_no[x3+x4]))
out = vals.fit(cov_type = 'HC0')
#print(out.params)
#print(out.summary())

x1 = pd.get_dummies(df['state'])
x2 = pd.get_dummies(df['year'])
df4 = pd.concat([df, x1, x2], axis=1)
data2 = df4[df4['years_post_prim']>-1]

data2['new'] = out.predict(sm.add_constant(data2[x3+x4]))
#print(data['new'])

data2['tai_i_t'] = data2['log_fatal_per_cap'] - data2['new']

print(data2['tai_i_t'])
print(data2['tai_i_t'].mean())
print(data2[data2['years_post_prim']==0]['tai_i_t'].mean())

'''yaz1 = pd.read_stata('q3b_ATTs.dta')
print(yaz1)
print(yaz1['ATT_h_Liu'].mean())'''

19     -0.005317
20     -0.006582
21     -0.005671
22     -0.025512
81     -0.151497
          ...   
940    -0.185011
941    -0.237544
942    -0.251788
1033   -0.187367
1034   -0.275325
Name: tai_i_t, Length: 182, dtype: float64
-0.10751347368607912
-0.05352726184482127
     h  ATT_h_Liu
0    0  -0.053527
1    1  -0.065068
2    2  -0.067590
3    3  -0.063771
4    4  -0.085229
5    5  -0.096785
6    6  -0.115518
7    7  -0.092711
8    8  -0.154099
9    9  -0.136518
10  10  -0.135313
11  11  -0.144515
12  12  -0.157780
13  13  -0.143960
14  14  -0.137506
15  15  -0.140737
16  16  -0.204112
17  17  -0.224800
18  18  -0.267387
19  19  -0.274797
-0.13808606564998627


<ipython-input-316-90a9cecaac63>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['new'] = out.predict(sm.add_constant(data2[x3+x4]))
<ipython-input-316-90a9cecaac63>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['tai_i_t'] = data2['log_fatal_per_cap'] - data2['new']


In [314]:
data_no_treat = df[df['years_post_prim']<0]
x1 = pd.get_dummies(data_no_treat['state'])
x2 = pd.get_dummies(data_no_treat['year'])
data_no = pd.concat([data_no_treat, x1, x2], axis=1)

x3 = sorted(list(set(list(data_no_treat['state']))))[1:]
x4 = sorted(list(set(list(data_no_treat['year']))))[1:]


vals = sm.WLS(data_no['log_fatal_per_cap'], sm.add_constant(data_no[x3+x4]), weights= data_no['population'])
out = vals.fit(cov_type = 'HC0')
#print(out.params)
#print(out.summary())

x1 = pd.get_dummies(df['state'])
x2 = pd.get_dummies(df['year'])
df4 = pd.concat([df, x1, x2], axis=1)
data2 = df4[df4['years_post_prim']>-1]

data2['new'] = out.predict(sm.add_constant(data2[x3+x4]))
#print(data['new'])

data2['tai_i_t'] = data2['log_fatal_per_cap'] - data2['new']

print(data2['tai_i_t'])
print(data2['tai_i_t'].mean())
print(data2[data2['years_post_prim']==0]['tai_i_t'].mean())

19     -0.001606
20      0.007319
21      0.008833
22     -0.005974
81     -0.148404
          ...   
940    -0.164698
941    -0.216627
942    -0.225838
1033   -0.159177
1034   -0.242101
Name: tai_i_t, Length: 182, dtype: float64
-0.09775911178488182
-0.04887688449106025


<ipython-input-314-b1c1db2ebbcc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['new'] = out.predict(sm.add_constant(data2[x3+x4]))
<ipython-input-314-b1c1db2ebbcc>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['tai_i_t'] = data2['log_fatal_per_cap'] - data2['new']
